In [2]:
import random
import time
import sys
from collections import Counter
import numpy as np
import grpc

import simulation_service_pb2 as simService
import simulation_service_pb2_grpc as simServiceGRPC

In [3]:
channel = grpc.insecure_channel('192.168.99.100:30224')
stub = simServiceGRPC.SimulationServiceStub(channel)

# Helper Functions #

In [41]:
def randomAction():
    rand = random.randint(1, 3)
    if rand == 1:
        return "MOVE"
    if rand == 2:
        return "CONSUME"
    if rand == 3:
        return "WAIT"
    
def randomDirection():
    rand = random.randint(1, 4)
    if rand == 1:
        return "UP"
    if rand == 2:
        return "DOWN"
    if rand == 3:
        return "LEFT"
    if rand == 4:
        return "RIGHT"

# Test Commands #

In [96]:
stub.ResetWorld(simulation_pb2.Empty())

In [8]:
agent = simService.Agent(id="", x=1, y=1)

In [9]:
spawnAgentRes = stub.CreateAgent(simService.CreateAgentRequest(agent=agent))
agentId = spawnAgentRes.id

In [10]:
print(agentId)
resp = stub.GetAgentObservation(simService.GetAgentObservationRequest(id=agentId))
print(resp)
cellList = list(resp.observation.Cells)
print(cellList)

dbf32040-b659-477e-b8ef-53a4bff77f96
api: "v1"
observation {
  Cells: "EMPTY"
  Cells: "AGENT"
  Cells: "EMPTY"
  Cells: "AGENT"
  Cells: "EMPTY"
  Cells: "EMPTY"
  Cells: "EMPTY"
  Cells: "EMPTY"
  Alive: true
  Energy: 100
  Health: 100
}

['EMPTY', 'AGENT', 'EMPTY', 'AGENT', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY']


In [11]:
cellList.insert(4, 'A')
cellListMatrix = np.array(cellList).reshape(3,3)
print(cellListMatrix)

[['EMPTY' 'AGENT' 'EMPTY']
 ['AGENT' 'A' 'EMPTY']
 ['EMPTY' 'EMPTY' 'EMPTY']]


In [6]:
actionRes = stub.AgentAction(simulation_pb2.AgentActionRequest(Id=agentId, Direction="UP", Action="CONSUME"))
actionRes.Successful

True

In [23]:
obsv = stub.AgentObservation(simulation_pb2.AgentObservationRequest(Id=agentId))
cellList = list(obsv.Cells)
cellList.insert(4, 'A')
cellMatrix = np.array(cellList).reshape(3,3)
print(cellMatrix)

[['EMPTY' 'EMPTY' 'EMPTY']
 ['FOOD' 'A' 'EMPTY']
 ['EMPTY' 'EMPTY' 'EMPTY']]


In [16]:
print(obsv.Energy)

105


In [16]:
stub.ResetWorld(simulation_pb2.ResetWorldRequest())

In [ ]:
for cellUpdate in stub.Spectate(simulation_pb2.SpectateRequest(Id="test-id")):
    print(cellUpdate)

In [ ]:
stub.SubscribeToRegion(simulation_pb2.SubscribeToRegionRequest(Id="test-id", X=1, Y=1))

# Training #

In [15]:
initial_games = 2
goal_steps = 500
score_requirement = 32
training_data = []
scores = []
accepted_scores = []
for episode in range(initial_games):
    game_memory = []
    # Spawn a new agent
    spawnAgentRes = stub.SpawnAgent(simulation_pb2.SpawnAgentRequest(X=0, Y=0))
    agentId = spawnAgentRes.Id 
    print(agentId)
    # Score (essentially the age of the agent)
    score = 0
    # Perform actions and observations
    for t in range(goal_steps):
        obsv = stub.AgentObservation(simulation_pb2.AgentObservationRequest(Id=agentId))
        if obsv.Alive == False:
            break
        action = randomAction()
        direction = randomDirection()
        actionRes = stub.AgentAction(simulation_pb2.AgentActionRequest(Id=agentId, Direction=direction, Action=action))
        score += 1
        game_memory.append( [list(obsv.Cells), action] )
        time.sleep(.20)
    
    if score >= score_requirement:
        accepted_scores.append(score)
        for data in game_memory:
            training_data.append(data)
    
    # Reset the world
    stub.ResetWorld(simulation_pb2.ResetWorldRequest())
    
    scores.append(score)

# training_data_save = np.array(training_data)
# np.save('saved_training_data.npy', training_data_save)

# print('Average accepted score: ', mean(accepted_scores))
# print('Median accepted score : ', median(accepted_scores))
# print(Counter(accepted_scores))
        

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Method does not exist."
	debug_error_string = "{"created":"@1551205943.250572000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Method does not exist.","grpc_status":5}"
>